# Google OR-Toolsを使った最適化
Google Optimization Tools (OR-tools) を使って、配送最適化問題をやってみる

In [16]:
import folium
import numpy as np
import osmnx as ox
import pandas as pd

import ortools_cvrptw as oc

l = oc.ShopData()
lawsons = pd.DataFrame(l.locations)

demands = pd.DataFrame(l.demands)
table = pd.concat([lawsons, demands], axis=1)

### 六本木近辺のローソン店舗を定義
座標、店舗名、店舗ごとの積載需要を定義。 時間指定は使わない。

In [17]:
table.columns = ['緯度','経度','店舗名','ノードNo','時間指定','積載需要']
table

,緯度,経度,店舗名,ノードNo,時間指定,積載需要
0,35.668177,139.739724,アークヒルズ店,1809615572,"(00:00, 00:00)",1
1,35.668096,139.732814,港赤坂九丁目店,251864110,"(00:00, 02:00)",19
2,35.664911,139.743147,城山トラストタワー,1760515775,"(00:00, 02:00)",21
3,35.671598,139.748326,合同庁舎第７号館,2162858601,"(00:00, 02:00)",6
4,35.657162,139.739235,東麻布三丁目,499193143,"(00:00, 02:00)",19
5,35.668501,139.748969,虎ノ門一丁目,1618521241,"(00:00, 00:40)",7
6,35.670014,139.733718,赤坂六丁目店,499189994,"(00:00, 02:00)",12
7,35.657856,139.727462,100 元麻布店,2207933301,"(00:00, 02:00)",16
8,35.657044,139.736167,麻布十番一丁目店,499192852,"(00:00, 02:00)",6
9,35.660346,139.723092,西麻布店,1655440289,"(00:00, 02:00)",16


### ひとまず地図にプロット

In [18]:
#地図にプロット

ROPPONGI = (35.663236, 139.732275)
G = ox.graph_from_point(ROPPONGI, distance=3000, network_type='drive', simplify=False)
map = folium.Map(location=ROPPONGI, tiles='cartodbpositron', zoom_start=14)
for j in l.locations:
    folium.Marker([j['lat'], j['lon']], popup=j['name']).add_to(map)

In [19]:
map

## 最適化を実行
コードが長すぎるので、外部ファイルで実行。結果を表示。

In [25]:
%run ortools_cvrptw.py

車両番号 0(red):
 [アークヒルズ店 積載数0+1 00:00着] --|886m|--> [城山トラストタワー 積載数1+21 00:03着] --|1368m|--> [新橋六丁目店 積載数22+21 00:27着] --|1927m|--> [東麻布三丁目 積載数43+19 00:53着] --|454m|--> [麻布十番一丁目店 積載数62+6 01:13着] --|1364m|--> アークヒルズ店 積載数(68) 01:22着
 - ルート距離　　: 5999.03m
 - ルート荷物数　: 68
 - ルート所要時間: 01時間22分22秒

車両番号 1(blue):
 [アークヒルズ店 積載数0+1 00:00着] --|1004m|--> [合同庁舎第７号館 積載数1+6 00:03着] --|584m|--> [虎ノ門一丁目 積載数7+7 00:10着] --|378m|--> [虎ノ門琴平点 積載数14+14 00:18着] --|1267m|--> アークヒルズ店 積載数(28) 00:35着
 - ルート距離　　: 3232.89m
 - ルート荷物数　: 28
 - ルート所要時間: 00時間35分35秒

車両番号 2(green):
 [アークヒルズ店 積載数0+1 00:00着] --|437m|--> [赤坂氷川公園前店 積載数1+8 00:02着] --|461m|--> [赤坂六丁目店 積載数9+12 00:11着] --|260m|--> [港赤坂九丁目店 積載数21+19 00:23着] --|1565m|--> [100 元麻布店 積載数40+16 00:46着] --|892m|--> [西麻布店 積載数56+16 01:04着] --|2035m|--> アークヒルズ店 積載数(72) 01:25着
 - ルート距離　　: 5648.88m
 - ルート荷物数　: 72
 - ルート所要時間: 01時間25分25秒

全ルート総距離: 14880.8m
全ルート総時間: 03時間23分23秒


In [26]:
map

それっぽくなった

## 時間指定をやってみる
No.5**「虎ノ門一丁目」**とNo.10**「赤坂氷川公園前店」** の時間指定を開始後20分以内に　（00:00、02:00）→（00:00、**00:20**）

In [64]:
l = oc.ShopData()
lawsons = pd.DataFrame(l.locations)

demands = pd.DataFrame(l.demands)
table = pd.concat([lawsons, demands], axis=1)

table.columns = ['緯度','経度','店舗名','ノードNo','時間指定','積載需要']
table

,緯度,経度,店舗名,ノードNo,時間指定,積載需要
0,35.668177,139.739724,アークヒルズ店,1809615572,"(00:00, 00:00)",1
1,35.668096,139.732814,港赤坂九丁目店,251864110,"(00:00, 02:00)",19
2,35.664911,139.743147,城山トラストタワー,1760515775,"(00:00, 02:00)",21
3,35.671598,139.748326,合同庁舎第７号館,2162858601,"(00:00, 02:00)",6
4,35.657162,139.739235,東麻布三丁目,499193143,"(00:00, 02:00)",19
5,35.668501,139.748969,虎ノ門一丁目,1618521241,"(00:00, 00:40)",7
6,35.670014,139.733718,赤坂六丁目店,499189994,"(00:00, 02:00)",12
7,35.657856,139.727462,100 元麻布店,2207933301,"(00:00, 02:00)",16
8,35.657044,139.736167,麻布十番一丁目店,499192852,"(00:00, 02:00)",6
9,35.660346,139.723092,西麻布店,1655440289,"(00:00, 02:00)",16


In [38]:
%run ortools_cvrptw.py 

車両番号 0(red):
 [アークヒルズ店 積載数0+1 00:00着] --|1110m|--> [虎ノ門一丁目 積載数1+7 00:03着] --|1298m|--> [新橋六丁目店 積載数8+21 00:13着] --|1355m|--> [城山トラストタワー 積載数29+21 00:38着] --|1434m|--> [麻布十番一丁目店 積載数50+6 01:02着] --|369m|--> [東麻布三丁目 積載数56+19 01:09着] --|1397m|--> アークヒルズ店 積載数(75) 01:31着
 - ルート距離　　: 6962.62m
 - ルート荷物数　: 75
 - ルート所要時間: 01時間31分31秒

車両番号 1(blue):
 [アークヒルズ店 積載数0+1 00:00着] --|807m|--> [港赤坂九丁目店 積載数1+19 00:02着] --|1565m|--> [100 元麻布店 積載数20+16 00:25着] --|892m|--> [西麻布店 積載数36+16 00:43着] --|1753m|--> [赤坂六丁目店 積載数52+12 01:04着] --|461m|--> [赤坂氷川公園前店 積載数64+8 01:17着] --|437m|--> アークヒルズ店 積載数(72) 01:26着
 - ルート距離　　: 5914.21m
 - ルート荷物数　: 72
 - ルート所要時間: 01時間26分26秒

車両番号 2(green):
 [アークヒルズ店 積載数0+1 00:00着] --|1004m|--> [合同庁舎第７号館 積載数1+6 00:03着] --|422m|--> [虎ノ門琴平点 積載数7+14 00:10着] --|1267m|--> アークヒルズ店 積載数(21) 00:27着
 - ルート距離　　: 2693.34m
 - ルート荷物数　: 21
 - ルート所要時間: 00時間27分27秒

全ルート総距離: 15570.17m
全ルート総時間: 03時間25分25秒


In [39]:
map

ルートに変化あった。
ルートの色が変わってしまってわかりづらいですが・・

数値結果

**時間指定なし**
全ルート総距離: 14880.8m
全ルート総時間: 03時間23分23秒

**時間指定あり**
全ルート総距離: 15570.2m
全ルート総時間: 03時間25分25秒

時間指定すると、少し増えた
